In [22]:
import pandas as pd
import numpy as np

from plotly import graph_objects as go

In [95]:
axes = ["x", "y", "z"]
pool_size = {"x": 360, "y": 180, "z": 70}
bins_by_ax = {ax: np.arange(0, s, 10) for ax, s in pool_size.items()}

In [94]:
behav = (
    pd.read_parquet("app_data/behav_sim.parquet")
    .clip(lower=0)
    .pipe(
        lambda df: df.assign(
            **{
                f"{ax}_position": df[f"{ax}_position"].clip(upper=s)
                for ax, s in pool_size.items()
            }
        )
    )
    .pipe(
        lambda df: df.assign(
            **{
                f"{ax}_bin": pd.cut(
                    df[f"{ax}_position"], bins=bins, include_lowest=True
                )
                .apply(lambda i: i.left)
                .astype(int)
                for ax, bins in bins_by_ax.items()
            }
        )
    )
)

In [77]:
behav.to_parquet("app_data/behav_sim.parquet")

In [80]:
from itertools import product

In [86]:
bin_cols = [f"{ax}_bin" for ax in axes]

In [111]:
X, Y, Z = np.mgrid[[slice(None, pool_size[ax] // 10 - 1) for ax in axes]]

In [24]:
pos_dict = {
    ax: behav.loc[coarse_indices, f"{ax}_position"].values for ax in ["x", "y", "z"]
}

In [25]:
go.Figure(
    data=[
        dict(
            **pos_dict,
            type="scatter3d",
            mode="lines",
            line=dict(color="black", width=0.5)
        )
    ]
)